In [1]:
import numpy as np

In [4]:
class Gaussian_total():
    def __init__(self,Gaussian_mean =[],Gaussian_var =[],Gaussian_weight =[],number_of_Gaussian = 0):
        self.Gaussian_mean =[]
        self.Gaussian_var =[]
        self.Gaussian_weight =[]
        self.number_of_Gaussian = number_of_Gaussian
class HMM_total():
    def __init__(initial_matrix=[], transition_cost =[], state_mix =[],state_number = 0 ):
        self.initial_matrix=initial_matrix
        self.transition_cost=transition_cost
        self.state_mix=state_mix

In [5]:
def gaussian(data,sigma_square,ftheta):
    sigma_square_producted=np.prod(sigma_square,axis=1)
    p=(1/(np.sqrt((2*np.pi)**2 *sigma_square)))*(np.exp(-0.5*np.sum((data-ftheta)**2/sigma_square_producted,axis=1)))
    return p
def log_gaussian(data,sigma_square,ftheta):
    cost= 0.5*np.sum(np.log((2*np.pi)*(sigma_square)),axis=1)+0.5*np.sum(np.square((ftheta-data))/sigma_square,axis=1)
    return cost
def mix_log_gaussian(mix_gaussian,ftheta):
    Gaussian_mean=mix_gaussian.Gaussian_mean
    Gaussian_var=mix_gaussian.Gaussian_var
    Gaussian_weight=mix_gaussian.Gaussian_weight
    cost=log_gaussian(Gaussian_mean,Gaussian_var,ftheta)
    changed_cost=np.sum(Gaussian_weight*cost)
    return changed_cost
def mix_gaussian(mix_gaussian,ftheta):
    cost=0
    for i in range(0,mix_gaussian.number_of_Gaussian):
        Gaussian_mean=mix_gaussian.Gaussian_mean[i][:]
        sigma_square=mix_gaussian.Gaussian_var[i][:]
        Gaussian_weight=mix_gaussian.Gaussian_weight[i]
        cost0=gaussian(Gaussian_mean,sigma_square,ftheta)
        cost=cost+Gaussian_weight*cost0
    return cost

In [ ]:
def GMMHMM_DTW(HMM_total,data):
    transition_cost=HMM_total[1]
    temp_list0=np.zeros([39])
    temp_list1=zp.zeros([39])
    